In [1]:
import glob
import numpy as np
import gpflow
import pandas as pd

from lineshape_meanfuncs import LineShape, Etalon, ComboMeanFunc
from gp_models import SwitchedGPR

import matplotlib.pyplot as plt

HAPI version: 1.1.0.9.6
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2017 are now available in HAPI code

           It is free to use HAPI. If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005


In [23]:
def lineshape_from_dataframe(frame, limit_factor_dict={}, line_kwargs={}):
    nu_list = frame.filter(regex="nu_*\d$").values.flatten().tolist()
    sw_list = frame.filter(regex="sw_*\d$").values.flatten().tolist()
    #Infer number of data sets from highest number associated with nu or sw
    n_dsets = len(nu_list)
    param_dict = {}
    vary_dict = {}
    constraint_dict = {}
    vary_dict['nu'] = bool(np.sum(frame.filter(regex="nu_*\d_vary$").values)) #Logical or, if any True, vary all
    vary_dict['sw'] = bool(np.sum(frame.filter(regex="sw_*\d_vary$").values, dtype=bool))
    #Loop over parameters in dataframe excluding nu and sw
    for name, val in frame.iteritems():
        if (name in ['molec_id', 'local_iso_id']) or ('err' in name) or ('sw' in name and not 'scale_factor' in name) or ('nu' in name and not 'VC' in name):
            continue
        else:
            new_name = name.replace('_air', '').lower()
            if 'y_296' in new_name:
                new_name = new_name.replace('y_296', 'y')
            elif 'sw_scale_factor' in new_name:
                new_name = new_name.replace('sw_scale_factor', 'sw_scale_fac')
            if 'vary' in name:
                vary_dict[new_name.replace('_vary', '')] = bool(val.values[0])
            else:
                param_dict[new_name] = val.values[0]
                try:
                    constraint_type, constraint_info = limit_factor_dict[name]
                    if constraint_type == 'magnitude':
                        constraint_dict[new_name] = (val.values[0] - constraint_info,
                                                     val.values[0] + constraint_info)
                    elif constraint_type == 'factor':
                        constraint_dict[new_name] = (val.values[0] / constraint_info,
                                                     val.values[0] * constraint_info)
                except KeyError:
                    pass
    #Create our lineshape function
    lineshape = LineShape(frame['molec_id'].values[0], frame['local_iso_id'].values[0],
                          n_data_sets=n_dsets,
                          nu=nu_list,
                          sw=sw_list,
                          **param_dict,
                          **line_kwargs,
                          constraint_dict=constraint_dict
                          )
    #Freeze somethings and let others vary
    for name, val in vary_dict.items():
        #try:
        gpflow.set_trainable(getattr(lineshape, name), val)
        #except AttributeError:
        #    pass
    return lineshape


In [24]:
lineparam_list = pd.read_csv('Parameter_LineList.csv', index_col=0)

In [25]:
line = lineshape_from_dataframe(lineparam_list[10:11],
                                limit_factor_dict={'SD_gamma':('factor', 2.0),
                                                   'nu':('magnitude', 0.1),
                                                   'sw':('limit', 2.0)},
                                line_kwargs={'cutoff':25})

In [26]:
line

name,class,transform,prior,trainable,shape,dtype,value
LineShape.nu,Parameter,Identity,,True,"(4,)",float64,"[13156.28028912, 13156.28029172, 13156.28030889..."
LineShape.sw,Parameter,Identity,,True,"(4,)",float64,"[4.73249416, 4.74980401, 4.74235728..."
LineShape.gamma0,Parameter,Identity,,True,(),float64,0.0448705149862343
LineShape.delta0,Parameter,Identity,,False,(),float64,-0.007967
LineShape.sd_gamma,Parameter,Identity,,True,(),float64,0.1123989299184388
LineShape.sd_delta,Parameter,Identity,,False,(),float64,0.0
LineShape.nuvc,Parameter,Identity,,False,(),float64,0.0
LineShape.eta,Parameter,Identity,,False,(),float64,0.0
LineShape.y,Parameter,Identity,,False,(),float64,0.0
